Chapter 1. Basic Expressions
=======

Expressions in **Prove-It** are fundamental and versatile.  In order to prove some statement to be true, you must be able to express the statement that you want to prove, and express each axiom and each theorem that is used to construct the proof as well as intermediate statements along the way.  
**Expressions** form **known truths** that are used in the derivation steps of a **proof** and will be discussed later.
An expression is a tree-like data structure (technically a directed acyclic graph, DAG, since there may be multiple instances of the same sub-expression).  As a Python object, its base class is `proveit._core_.expression.expr.Expression` (also aliased as `proveit.Expression` via "`from ... import ...`" Python statements within `__init__.py` files):

In [ ]:
from proveit import Expression
%begin basic_expressions
Expression

An `Expression` object must be one of the "core" `Expression` classes listed below that is each derived from `proveit._core_.expression.expr.Expression`.  The list is reproduced from our <a class="ProveItLink" href="../packages/brief_guide.ipynb">brief guide</a> but more details and some code examples are provided below in this tutorial.  The "basic" types of expressions that are covered in this chapter are:
* <a href="#Variable">`Variable`</a>: A label that is interchangeable (as long as it is kept distinct from other labels) with no intrinsic meaning.  It is usually represented by a single letter but can have any representation.
* <a href="#Literal">`Literal`</a>: A label that is not interchangeable and has an intrinsic meaning.  Specific operators ($\lnot, \land, +, \times$, etc.) and specific irreducibe values ($\top, \bot, 0, 5$, etc.) are all `Literal`s.  Furthermore, a problem-story `Variable` in a particular **context**, representing some unknown but particular value, should also be a `Literal` (e.g., "Ann has $a$ apples...").
* <a href="#Operation">`Operation`</a>: The application of *operator(s)* on *operand(s)*.  For example, $0 + 5 + 8$ and $1 < a \leq b < 3$ are examples of *operation* expressions.  The **Prove-It** library defines many *types* derived from the *operation* type (e.g., for each specific operation), but the <a href="#proof_deriv">derivation rules</a> only need to know that these are *operations*.
* <a href="#Lambda">`Lambda`</a>: A mapping defined by *parameter* `Variable`s transforming to some *body* `Expression` but only when the *parameter(s)* meet certain *condition(s)*.  For example, $(x, y, z) \mapsto x+y/z~|~x \in \mathbb{R}, y \in \mathbb{R}, z \in \mathbb{R}, z \neq 0$ is a conditional **lambda** that converts three real numbers $x, y, z$ to $x + y/z$ as long as $z$ is not zero.  Note that a `Lambda` introduces `Variables` into a new scope via the *parameters*.  These *parameter* `Variable`s are said to be *bound* in this new scope; occurrences outside this scope are not deemed to be the same thing. There is a special category of `Operation` types called `OperationOverInstances` that are essentially functionals acting on a `Lambda` operand.  $\forall$, $\exists$, $\sum$, and $\prod$ are examples of `OperationOverInstances`.  For example, $\forall_{x~|~Q(x)} P(x)$ is an expression that translates to "$P(x)$ is true for all values of $x$ for which $Q(x)$ is also true".  Internally, this is represented as an $\forall$ operator acting on the conditional `Lambda` map $x \mapsto P(x)~|~Q(x)$.
* <a href="#ExprList">`ExprList`</a>: A list of `Expression`s that is used, for example, when there are multiple *operators* or *operands* of an `Operation` or multiple *parameters* of a `Lambda`.
* <a href="#NamedExprs">`NamedExprs`</a>: A mapping from keyword strings to `Expression`s.  This can be used to prevent ambiguity of an expression's internal representation.

Advanced expressions covered in a <a href="tutorial10_advanced_expr.ipynb">later chapter</a>:
* <a href="tutorial10_advanced_expr.ipynb#ExprTensor">`ExprTensor`</a>: A multi-dimensional (e.g., two-dimensional) array of `Expression`s. This can be used, for example, to represent a quantum circuit which is a two-dimensional graphical representation of a sequence of quantum operations.  A 2-D representation of a matrix is a more basic example.  More work is required to properly implement and test the *ExprTensor* class.
* <a href="tutorial10_advanced_expr.ipynb#Indexed">`Indexed`</a>: A particular indexed element of a `Variable` where the `Variable` is intended to serve as a placeholder for an `ExprList` or `ExprTensor`.  It has a *base* which determines the indexing offset, typically $0$ or $1$.  For example, $x_5$ represents the fifth or sixth element of $x$ in *base* $1$ or $0$ respectively.  The *base* is typically not displayed and must be inferred by the context, though it is explicitly revealed in the <a href="#expr_info">expression information page</a>.  An `Indexed` expression is typically contained within an `Iter` (described next).
* <a href="tutorial10_advanced_expr.ipynb#Iter">`Iter`</a>: Represents an iteration of a *parameter* going from a *start* to an *end* in successive unit increments ($+1$).  For example, $x_1 +~\ldots~+ x_n$ contains an `Iter` of `Indexed` `Variable`.  If we take $n$ to be $3$, this would expand to $x_1 + x_2 + x_3$.  An `Iter` has a *lambda map*, *start index(indices)*, and *end index(indices)*.  In our example, the `Iter` $x_1,~\ldots,x_n$, is the *operand* of an `proveit.number.addition.Add` `Operation`.  The *lambda map*, *start index* and *end index* of the *iteration* are $i \mapsto x_i$, $1$, and $n$, respectively.


Developers and users may derive new classes from core `Expression` classes (`Literal` and `Operation` classes in * <a href="tutorial10_advanced_expr.ipynb#ExprTensor">`ExprTensor`</a>: A multi-dimensional (e.g., two-dimensional) array of `Expression`s. This can be used, for example, to represent a quantum circuit which is a two-dimensional graphical representation of a sequence of quantum operations.  A 2-D representation of a matrix is a more basic example.  More work is required to properly implement and test the *ExprTensor* class.particular) to make their own `Expression` classes that have special formatting rules for displaying the **expression** and methods for manipulating and utilizing the expression (i.e., applying theorems or axioms as will be discussed in later tutorial chapters).  Ideally, the formatted should be in direct correspondence with the structure of the `Expression` as a true reflection of the internal representation, but there is no enforcement mechanism to ensure that this is the case.  Users are advised to inspect the fully explicit expression DAGs of important axioms and theorems (or rely on crowdsource checking).  Conveniently, clicking on any **expression** rendered as LaTeX will link to a notebook that, when executed, will reveal this internal structure.  (Alternatively, the `exprInfo()` method may be called).

The remainder of this tutorial chapter will show examples and discuss details of the different core `Expression` classes.

As we discuss these different core types we will discuss the instances when **expressions** are considered to be the same or different.  This is important because it corresponds to whether or not an **expression** is the same as one that has been proven to be a true statement (**known truth**).

We will also discuss *expression substitution/relabeling* as we go through the different cases.  The following are related but distinct manipulations in **Prove-It**:
* *Expression substutition*: creating a new **expression** from an existing **expression** by swapping one or more sub-expression(s) for other sub-expression(s) by using the `substituted` method.  This creates a new `Expression` but has nothing to do, intrinsically, with proving statements.
* *Expression relabeling*: a restricted version of *expression substitution* in which any `Variable` may be relabeled only with another `Variable` by using the `relabeled` method.
* *Specialization* derivation step: deriving a statement of the form $P(y)$ from a statement of the form $\forall_x P(x)$.  Details of this will be discussed in a later tutorial chapter, but this manipulation uses *expression substitution* in generating $P(y)$.  However, specific rules and limitations apply to *specialization* that may not apply to *expression substitution* to ensure that the derivation is sound.  *Specialization* specifically involves statements (**known truths**) with the $\forall$ quantifier (Forall).
* *Relabeling* derivation step: deriving a statement (known truth) from another statement which differs only in its `Variable`s.  For example, from $\forall_x P(x)$, we could use *relabeling* to derive $\forall_y P(y)$.
* *Equality substitution*: using the `substitution` axiom of `proveit.logic.equality` to prove that $f(x) = f(y)$ given $x = y$.  This uses *specialization* of the `substitution` axiom which state $\forall_{f, x, y~|~x=y}~(f(x) = f(y))$.  This specifically involves statements (**known truths**) with the `proveit.logic.equality.equals._operator_` **Literal** (the `=` sign). This will be discussed in a later tutorial chapter.

The focus here will be *expression substitution/relabeling* with some mention of the additional restrictions applicable to the *specialization/relabeling* derivation steps that will be discussed in detail in later tutorial chapters.  It is important to understand the distinction of these types of manipulations.

<a name="#Label"></a>Labels (Variables and Literals)
================================

In [ ]:
from proveit import Label

The `Variable` and `Literal` classes both derive from `Label`.  A `Label` is created with a string and LaTeX format to determine how it is displayed.  In a Jupyter notebook, the LaTeX is rendered and presented in the output:

In [ ]:
tri = Label(stringFormat='triangle', latexFormat=r'\triangle')

After importing anything from `proveit` in a Jupyter notebook, whenever assignments are made at the end of an input cell the output shows the name of the assigned Python variable (not to be confused with a **Prove-It** *Variable*) followed by ':' and its rendered value.  In the above input, we have assigned `tri` to a `Label` object we have created which renders as a triangle shape.  For convenience, this Python variable name and shape appear in the output without any additional code.  The string format is presented when the object is converted to a string and is a useful alternate representation (used, for example, in error messages that are displayed as strings).

In [ ]:
str(tri)

The breakdown of an **expression** into its DAG structure is displayed by calling `exprInfo()`.  Clicking on the **expression** is also a way to view this information, linking to a page that shows a canonical way to build the **expression** and calling `exprInfo()` on that built **expression**.  Here we see that the `tri` is a `Label` with no sub-expressions.

In [ ]:
tri.exprInfo() # the expressions and sub-expressions are numbered with the top level being zero.

`Label` was not one of the core **expression** types listed above because it is not intended to be used except as the base class of `Variable` and `Literal`.  Notwithstanding, `Label` is displayed as the core type.
Two Label's are regarded to be the same expression iff both formats, and the class, are the same:

In [ ]:
assert tri == Label('triangle', r'\triangle') # equal when both formats are the same
assert tri != Label('tri', r'\triangle') # not equal when either is different

Let's make another label to test out substituted and relabeled methods:

In [ ]:
sq =  Label(stringFormat='square', latexFormat=r'\Box')

Using *expression substitution*, we can change one `Label` into another or any other `Expression`.  We'll see that *specialize* is more restrictive; only applicable to `Variable`s and not any `Label`, but for *expression substition* this is fine.

In [ ]:
tri.substituted({tri:sq})

<a href="#Variable"></a>Variable
===========
A `Variable` is an interchangeable <a href="#Label">Label</a> with no formal, contextual meaning.

In [ ]:
from proveit import Variable
x = Variable('x') # the string and latex formats are the same by default

In [ ]:
omega = Variable(stringFormat='omega', latexFormat=r'\omega') # different string and latex format

Convenient `Variable`s and other `Expression`s are accessible from `_common_.py` modules (generated from `_common_.ipynb` notebooks).  That is, `_common_.py` modules hold common expressions.  For example, from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit.common</a> we can import the following.

In [ ]:
from proveit._common_ import a, b, c, x, y, z, alpha
from proveit import ExprList
ExprList(a, b, c, x, y, z, alpha)

We used an <a href="#ExprList">ExprList</a> to conveniently render all of these `Variable`s as one `Expression`.  Expression information is similar to the `Label`, but with the different core type and class:

In [ ]:
alpha.exprInfo()

Just as we saw for the generic `Label` `Expression`s, we can use *expression substitution* to exchange one `Variable` for another or any other `Expression` or *expression relabeling* to exchange one `Variable` for another `Variable`.  Furthermore, we will see in a later tutorial chapter that `Variable`s have special properties with respect to *specialization* because they are defined as interchangeable labels.

In [ ]:
alpha.substituted({alpha:sq}) # we can substitute a Variable for any other Expression

In [ ]:
alpha.relabeled({alpha:omega}) # we can relabel a Variable to another Variable

In [ ]:
from proveit import ImproperRelabeling
try:
    alpha.relabeled({alpha:sq}) # we can NOT relabel a Variable to a non-Variable
    assert False, "Expecting an ImproperRelabeling error; should not make it to this point"
except ImproperRelabeling as e:
    print("EXPECTED ERROR:", e)

<a name="#Literal"></a>Literal
-------

A `Literal` is another kind of <a href="#Label">Label</a>.  In contrast to `Variable`s, `Literal`s have a formal, contextual meaning.

In [ ]:
from proveit import Literal

Below are different scenarios in which a `Literal` could appropriately be used.

### Specific operators

In [ ]:
TIMES = Literal(stringFormat='*', latexFormat=r'\times')

In [ ]:
FACTORIAL = Literal('!')

In [ ]:
SUMMATION = Literal('sum', r'\sum')

We will see in the <a href="#Operation">Operation</a> section that the *operator* of an `Operation`-derived class should be stored as a Python class variable called `_operator_`.  We are using module-level Python variable names just for these examples.

### Constant values

In [ ]:
FIVE = Literal('5') # Literal constants are also ALL-CAPS

In [ ]:
TRUE = Literal('true', r'\top')

In [ ]:
FALSE = Literal('false', r'\bot')

This will typically be defined as **common expressions** in the appropriate **context**, to be discussed in a later tutorial.

### Contextual "variables"

This is appropriate when variables are given a specific meaning within the context of a problem.  For example, consider a math "story problem" where Andrea has $a$ apples and Bill has $b$ bananas.  These are variables in a sense, but here they have contextual meaning so they should be `Literal`s.  However, suppose one wants to prove a general theorem for any number of Andrea's apples and Bill's bananas.  One may start by using the $a$ and $b$ `Literal`s for convenience for some contextual "lemmas" (as an aside, though, **Prove-It** does not distinguish between lemmas and theorems), but then demote them to `Variable`s in order to make a statement of the form $\forall_{a, b} P(a, b)$ (`Literal`s may not be quantified over).  This is done by using a process called axiom elimination that will be discussed later.  For now, we just note that "variables" with contextual meaning need to be `Literal`s.

In [ ]:
aLit = Literal('a')

In [ ]:
bLit = Literal('b')

In [ ]:
assert a != aLit # different because one is a Variable and one is a Literal
assert a == aLit.asVariable() # same after converting the Literal to a Variable

### Literal substitution
It is possible to perform *expression substitution* on a `Literal`.

In [ ]:
aLit.substituted({aLit:bLit})

In [ ]:
aLit.substituted({aLit:omega})

However, we cannot *relabel* a `Literal` since a **literal** is not an interchangeable label.  Furthermore, we will see in a later chapter that *specialization* is not allowed because you cannot quantify over a `Literal` like you can a `Variable`.

In [ ]:
try:
    aLit.relabeled({aLit:omega}) # we cannot relabel Literals
    assert False, "Expecting a TypeError; should not make it to this point"
except TypeError as e:
    print("EXPECTED ERROR: ", e)

### Literal contexts

A `Literal` is distinguished not only by its formatting but also by its **context**.  Axioms and theorems of **Prove-It** are organized via **contexts** which we will discuss later in more detail.  Each `Literal` belongs to a **context** which is typically the one corresponding to the package (directory) in which it is defined.  This is particularly important for *contextual variable* types of `Literal`s that are very *context*-specific.

In [ ]:
import proveit.logic as logic

In [ ]:
logic.TRUE

*If the above output is not the $\top$ symbol when you execute this, you probably need to <a class="ProveItLink" href="tutorial00_introduction.ipynb#build">build or download the Prove-It database</a>.*

In [ ]:
# When the context is different, they are not the same.
TRUE == logic.TRUE

The detailed expression information reveals the differences:

In [ ]:
TRUE.exprInfo(details=True)

In [ ]:
logic.TRUE.exprInfo(details=True)

**Expressions** are also distinguished by their class (derived from the *core type*).  In the above example, `logic.TRUE` is an object of the `proveit.logic.boolean.booleans.TrueLiteral` class.  `logic.TRUE` and `logic.FALSE` have an `evalEquality(..)` method defined in their respective classes which is convenient method for deriving any of the following: $(\top = \top) = \top$, $(\bot = \bot) = \top$, $(\top = \bot) = \bot$, or $(\bot = \top) = \bot$ (these are proven theorems within `proveit.logic`).

In [ ]:
logic.TRUE.evalEquality(logic.FALSE)

This is a sneak preview of a known truth (**KnownTruth** object) that uses the turnstile notation, $\boldsymbol{\vdash}$, to indicate that the expression is a proven statement.  There will be more on this in later tutorial chapters.

<a name="Lambda"></a>Lambda
====

This type of **expression** represents a mathematical mapping or function.  It contains *parameter(s)* (one or more `Variable`s and may also include `Iteration`s of `Indexed` `Variable`s that will be discussed in the <a href="tutorial10_advanced_expr.ipynb#Iteration">Iteration</a> section of the <a href="tutorial10_advanced_expr.ipynb">advanced expressions</a> chapter), a *body* (any `Expression` that the parameters are to be mapped into), and optional *conditions* (any `Expression`s).  The mapping is only defined when the *conditions* are satisfied (derivable as **known truths**).  Below is an example with a single *parameter* and no *conditions*, mapping any input to the $5$ **Literal**:

In [ ]:
from proveit import Lambda

In [ ]:
# maps any value to 5
mapTo5 = Lambda(x, FIVE) 
mapTo5

In [ ]:
# Let's look at the Expression info
mapTo5.exprInfo()

Here is an example with multiple *parameters* and *conditions* that are represented by `ExprList`s.  This maps a pair of real numbers to the first element (invoking the `InSet` <a href="#Operation">Operation</a>):

In [ ]:
# Can have multiple arguments
from proveit.logic import InSet
from proveit.number import Reals
lambdaExpr = Lambda([x, omega], x, conditions=[InSet(x, Reals), InSet(omega, Reals)])
lambdaExpr

In [ ]:
# Expression info shows how the Lambda is broken down into arguments and the expression as sub-expression.
lambdaExpr.exprInfo()

### Lambda substitution/relabeling

A very important property of a `Lambda` is that it defines a new "scope" for its parameters.  The parameter **Variable**s may be *relabeled*, but they will not be substituted.

In [ ]:
lambdaExpr.relabeled({omega:y})

In [ ]:
lambdaExpr.substituted({x:aLit})

We can substitute a Variable that is not an argument, however:

In [ ]:
mapTo5.substituted({FIVE:aLit})

There are important limits regarding *relabeling* and *substitution* within the "scope" of the `Lambda`.  In particular, parameters of the Lambda function are "reserved" and may not be used in *substitution* or *relabeling*.  Otherwise, you could alter the meaning of the **expression** in ways that can invalidate a proof (i.e., via a *specialization* step which relies upon *expression substitution* internally).

In [ ]:
from proveit import ScopingViolation
try:
    Lambda([x, omega], z).substituted({z:omega})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR: ", e)
try:
    Lambda([x, omega], z).relabeled({z:omega})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR: ", e)

In addition, Lambda parameters must be distinct.

In [ ]:
from proveit import ImproperSubstitution
try:
    Lambda([x, omega], z).relabeled({x:omega})
    assert False, "Expecting an ImproperSubstitution error; should not make it to this point"    
except ImproperSubstitution as e:
    print("EXPECTED ERROR: ", e)

It is possible, however, to perform simultaneous *relabeling* that is consistent (retains the meaning).

In [ ]:
lambdaExpr.relabeled({x:omega, omega:x})

<a name="Operation"></a>Operation
=========

This type of **expression** represents an applied operation.  It contains *operator(s)* (one or more `Label`s and may also include `Iteration`s of `Indexed` `Label`s that will be discussed in the <a href="tutorial10_advanced_expr.ipynb#Iteration">Iteration</a> section of the <a href="tutorial10_advanced_expr.ipynb">advanced expressions</a> chapter) and *operand(s)* (one or more of any `Expression`).  The default formatting of an `Operation` with one *operator* and more than one *operand* is to place the *operator* between each successive pair of *operands* as follows.

In [ ]:
from proveit import Operation
Operation(TIMES, [x, y, omega])

A `Function` is derived from `Operation`.  It behaves in the same manner except it formats it in a "function" style:

In [ ]:
from proveit import Function

# Simple case: a Variable operator with a single Variable operand
f = Variable('f')
fx = Function(f, x)
fx # f(x)

Common forms of these are available from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit.common</a>:

In [ ]:
from proveit._common_ import fx, gx, Px, Qx
ExprList(fx, gx, Px, Qx)

Let's derive a couple of classes from `Operation` to represent operations with pecific literal operators.
This is commonly done within Prove-It library modules.  We use the `%load` magic command to display (and execute) the contents of <a href="demo_operations.py">demo_operations.py</a>.

In [ ]:
# %load demo_operations.py
'''
Module that defines a Factorial Operation purely for purposes of a
tutorial.
'''

from proveit import Operation, Literal

class Factorial(Operation):
    # _operator_ is a special class variable name, defining specific literal operator of the Operation class.
    # It is not only used for default formatting but also when performing substitutions for rebuilding expressions.
    _operator_ = Literal('!')
    
    def __init__(self, operand):
        # creates the Operation with FACTORIAL as the operator and the provided operand as its only operand.
        Operation.__init__(self, Factorial._operator_, operand) # initializes self.operand

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.string(fence=True) + Factorial._operator_.string()
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.latex(fence=True) + Factorial._operator_.latex() 

class Multiply(Operation):
    
    # This operator Literal has a LaTeX format that differs from the string format.
    _operator_ = Literal('*', r'\times')
    
    def __init__(self, *operands): # takes a list of arguments as the operands
        # creates the AssociativeOperation with TIMES as the operator and any number of operands.
        Operation.__init__(self, Multiply._operator_, operands)
    
    # The default formatting will display the operator between the operands


In order to work properly, however, we need to import these classes from the module (as Prove-It will use the `__file__` attribute of this module for its internal purposes).

In [ ]:
from demo_operations import Factorial, Multiply

A `Factorial` is an `Operation` with "!" as the format of the `Literal` operator.  Its formatting is altered from the default but is still true to the core `Expression` structure (the formatting is a reflection of the internal structure).

In [ ]:
# Now we can make an object with this new class
xFactorial = Factorial(x)

In [ ]:
# show core structure
xFactorial.exprInfo()

In the Jupyter notebook setting, we typically only use LaTeX formatting, but it is important to define the string formatting as well for whenever it is needed (e.g., when error messages are displayed).  The `fence=True` above indicates that parentheses should be used when it could be ambiguous otherwise.  Our example above has no ambiguity, so parentheses are not used.  We will show a case below where the parentheses are required.

Next we consider a multiple *operand* example using the `Multiply` class that we imported from <a href="demo_operations.py">demo_operations.py</a> above.

In [ ]:
# Demonstrating an Operation with multiple operands
multExpr = Multiply(x, FIVE, omega)

In [ ]:
# Let's nest Operations and show proper fencing behavior
nestedOperation = Factorial(multExpr)

In [ ]:
nestedOperation.exprInfo()

### Operand substitution/relabeling

*Substitution* and *relabeling* of *operands* is straightforward in the way it works.

In [ ]:
xFactorial.relabeled({x:omega})

In [ ]:
nestedOperationFromSub = xFactorial.substituted({x:multExpr})

Internally, however, there is specific machinery required to regenerate these objects and construct them with the proper classes.  In order for this machinery to work, the `_operator_` class attribute must be properly defined (e.g., `Factorial._operator_` and `Multiply._operator_`).  To demonstrate that this is working properly, note that the classes displayed in the detailed expression info are as they should be:

In [ ]:
nestedOperationFromSub.exprInfo(details=True)

Furthermore, this expression that we obtained via substitution is equivalent to the one constructed directly:

In [ ]:
print("Expressions, generated in different ways, are the same:", (nestedOperationFromSub == nestedOperation))

### Operator substitution/relabeling

The *operator* may likewise be *substituted* or *relabeled*, but with some interesting extra capabilities.

It is straightforward to *relabel* a **Variable** *operator* with another **Variable**.

In [ ]:
from proveit._common_ import g
fx.relabeled({f:g}) # Variable operator to a different Variable

And it is straightforward to *substitute* a **Variable** *operator* with a **Literal**.  This is generally not desirable, however, because it may not have the appropriate class.

In [ ]:
xFactorial_from_fx = fx.substituted({f:FACTORIAL})

In [ ]:
xFactorial_from_fx.exprInfo(details=True)

In [ ]:
assert xFactorial_from_fx == xFactorial # Same

It is able to produce generate the new **expression** in the appropriate `Factorial` class via the internal mechanisms that use its `_operator_` class attribute.

Performing an *expression substitution* of a `Literal` *operator* within an `Operation` class will typically be blocked because it will not know how to make the new **expression**:

In [ ]:
from proveit import OperationError
# But this will typically be prevented (appropriately) in trying to remake a derived Operation class:
try:
    xFactorial.substituted({FACTORIAL:SUMMATION})
    assert False, "Expecting an OperationError error; should not make it to this point"
except OperationError as e:
    print("EXPECTED ERROR:", e)

### Operation substitution

The more interesting and useful case is to *substitute* the `Operation` itself.

In [ ]:
fx.substituted({f:Lambda(y, Factorial(y))})

Note that the operand is still $x$ (not $y$).  This is substituting the operation, not the operand.  If desired, however, it could be substituted simultaneously:

In [ ]:
omegaFactorial = fx.substituted({f:Lambda(y, Factorial(y)), x:omega})
omegaFactorial

In [ ]:
omegaFactorial.exprInfo(details=True)

Note that by substituting $f$ with a `Lambda` expression, the entire `Operation` is substituted, not just the *operator*.  This is why `Lambda` *operator*s (only `Label` operators) are not allowed; otherwise, such a substitution could be ambiguous or confusing (should it substitute the `Operation` or substitute the *operator* with the `Lambda` expression?).  This also highlights the fact that **Prove-It** is not a functional programming or lambda calculus.  Rather, **Prove-It** is designed for manipulating expressions as desired, and *operation substitution* is a useful and powerful, yet relatively straightforward tool for doing so.

### Mocking up **Lambda** operators

It would not be consistent with the **Prove-It** philosophy of *freedom of expression* to completely disallow expressions involving a `Lambda` function applied to operand(s).  While having a `Lambda` *operator* for an `Operation` is not allowed, we are free to make an `Operation` class whose *operands* are the `Lambda` function as well as the operand on which we wish to apply the function. As a demonstration, we define such an `Operation`, called `LambdaApplication`, in <a href="demo_lambda_app_operation.py">demo_lambda_app_operation.py</a>.

In [ ]:
# %load demo_lambda_app_operation
'''
Module that defines a LambdaApplication Operation
class for demonstration purposes in this tutorial.
'''

from proveit import Operation, Literal, NamedExprs

class LambdaApplication(Operation):
    _operator_ = Literal('LAMBDA_APPLICATION', r'{\rm LAMBDA\_APPLICATION}')
    
    def __init__(self, lambdaFn, operand):
        Operation.__init__(self, LambdaApplication._operator_, NamedExprs([('lambdaFn',lambdaFn), ('operand',operand)]))
        self.lambdaFn = self.operands['lambdaFn'] # The Lambda function operand
        self.lambdaOperand = self.operands['operand'] # The operand of the Lambda function

    @classmethod
    def extractInitArgValue(operationClass, argName, operator, operand):
        '''
        Given a name of one of the arguments of the __init__ method,
        return the corresponding value as determined by the operator and
        operand of the LambdaApplication Operation.
        (This is important so that Prove-It knows how to 'make' an altered
        copy of this Operation).
        '''
        assert isinstance(operand, NamedExprs), "Expecting LambdaApplication operand to be a NamedExprs object"
        if argName=='lambdaFn': 
            return operand['lambdaFn']
        elif argName=='operand':
            return operand['operand']

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.string(fence=True) + '(' + self.lambdaOperand.string() + ')'
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.latex(fence=True) + '(' + self.lambdaOperand.latex() + ')'


In [ ]:
from demo_lambda_app_operation import LambdaApplication

In [ ]:
lambdaAppExpr = LambdaApplication(Lambda(y, Factorial(y)), x)

Note that the `_operator_` of the `LambdaApplication` s not displayed but is implicit in the context of the lambda application formatting.  Also, we made use of `NamedExprs` which will be discussed next.  Let us take a look at the expression information.

In [ ]:
lambdaAppExpr.exprInfo(details=True)

**Axioms** could then be created to define how the `LambdaApplication` **operation** actually operates, but **axioms** will be discussed in a later chapter.

NamedExprs
=========

`NamedExprs` is a composite **expression** that maps string keywords to sub-**expressions**.  The reason that this may be desired is to be explicit in the internal representation about the role of each sub-**expression** so there is no ambiguity between the internal and external representation.  A good example is `lambdaAppExpr` above where one *operand* plays the role of the lambda function and the other *operand* plays the role of the lambda function's operand.

Here is a more rudimentary demonstration of using **NamedExpressions**:

In [ ]:
from proveit import NamedExprs
NamedExprs([('left',x), ('right',y), ('product',Multiply(x, y))])

To produce a `NamedExprs` object, you must supply a list of (keyword, `Expression`) pairs.  The keywords must be strings.  When the **NamedExprs** is displayed, it will show each of these pairs in the order that they were originally provided.

<a name="ExprList"></a>ExprList
====================

We have seen examples of `ExprList`s above.  On its own, it is simple.  It becomes more interesting when we discuss `Iter`s below.

In [ ]:
from proveit import ExprList
xy = ExprList(x, y)

In [ ]:
xyz = ExprList(x, y, z)

Substitution is straightforward:

In [ ]:
fancyList = xyz.substituted({z:nestedOperation})
fancyList

In [ ]:
# Let's keep it up
fancierList = fancyList.substituted({x:nestedOperation})
fancierList

Nesting `ExprList`s is possible:

In [ ]:
ExprList(x, ExprList(y, z))

It is also useful to note that, as a convenience, python lists of **expressions** are typically transformed automatically into an `ExprList` when they are passed into **Prove-It** methods that accept or expect them for that argument.  For example, the following generates the same **expression** as above.

In [ ]:
ExprList(x, [y, z])

<a name="AdvancedExprs"></a>Advanced Expressions
====================

More advanced types of expressions will be discussed in a <a href="tutorial10_advanced_expr.ipynb">later chapter</a>.  These advanced expressions allow multi-dimensional representations of mathematical structure (for example, explicit matrix representations or quantum circuits) via the <a href="tutorial10_advanced_expr.ipynb#ExprT?ensor">`ExprTensor`</a> type as well as iterated ranges within lists or tensors via the <a href="tutorial10_advanced_expr.ipynb#ExprTensor">`Iter`</a> type (e.g., $x_1 +~\ldots~+ x_n$).  These advanced expression types are useful in forming explicit representations commonly used to communicate mathematical concepts.  They are not necessary, however, in general theorem proving.  One could choose not to use such constructs and only represent such concepts via implicit representations and recursion.  Following the **Prove-It** philosophy, however, we want to allow the use of explicit representation tied as directly as possible into our formal proofs.  But we will skip these for now so we can cover our theorem proving concepts without this extra complication.

In [ ]:
%end basic_expressions

# Next chapter: <a class="ProveItLink" href="tutorial02_proof_basics.ipynb">Proof Basics</a>

## <a class="ProveItLink" href="tutorial00_introduction.ipynb#contents">Table of Contents</a>